In [3]:
!pip3 install pyspellchecker
!pip3 install nltk
!pip3 install gensim

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
import warnings
import gensim
import gensim.downloader
warnings.simplefilter("ignore", category=RuntimeWarning)
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to /Users/banji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
class Preprocessing:
    def __init__(self, path_to_data):
        files = []
        tweets = pd.DataFrame()
        for filenames in os.listdir(path_to_data):
            if re.findall('.+\.csv', filenames):
                files.append(filenames)
        for file in files:
            with open(path_to_data + '/' + file, 'r') as f:
                data = f.read()
            data = re.sub(r',(?=[^ ])', '--', data)
            df = pd.DataFrame(data.split('--'), columns=['tweets'])
            df['category'] = file
            df.drop_duplicates(inplace=True)
            tweets = tweets.append(df)
        tweets['category'].replace({
            'processedNegative.csv' : -1, 
            'processedNeutral.csv': 0, 
            'processedPositive.csv' : 1}, inplace=True)
        tweets.index = np.arange(len(tweets))
        self.tweets = tweets
        self.spell = SpellChecker()

    def tokenization(self):
        array = []
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
        self.tweets["just_tokenization"] = array

    def PorterStemmer(self, misspellings=False):
        array = []
        porter_stemmer = PorterStemmer()
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
            for i, w in enumerate(array[-1]):
                if (misspellings == True):
                    array[-1][i] = self.spell.correction(w)
                array[-1][i] = porter_stemmer.stem(w)
    
        if (misspellings):
            self.tweets["PorterStemmer+misspellings"] = array
        else:
            self.tweets["PorterStemmer"] = array

    def WordNetLemmatizer(self, misspellings=False):
        nltk.download('wordnet')
        array = []
        lemmatizer = WordNetLemmatizer()
        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
            for i, w in enumerate(array[-1]):
                if (misspellings == True):
                    array[-1][i] = self.spell.correction(w)
                array[-1][i] = lemmatizer.lemmatize(w)
                    
        if (misspellings):
            self.tweets["WordNetLemmatizer+misspellings"] = array
        else:
            self.tweets["WordNetLemmatizer"] = array

    def good_preproc(self):
        array = []
        porter_stemmer = PorterStemmer()
        nltk.download('stopwords')
        english_stopwords = stopwords.words('english')

        for sent in self.tweets.tweets:
            array.append(word_tokenize(sent))
            for i, w in enumerate(array[-1]):
                array[-1][i] = porter_stemmer.stem(w)
                if (len(array[-1][i]) < 3):
                    array[-1].pop(i)
                elif (array[-1][i] in english_stopwords):
                    array[-1].pop(i)
                elif (array[-1][i].isdigit()):
                    array[-1].pop(i)
        self.tweets["good_preproc"] = array

In [27]:
#минут 6 учицца - норма

preproc = Preprocessing("./data")
preproc.tokenization()
preproc.PorterStemmer(False)
preproc.WordNetLemmatizer(False)
preproc.WordNetLemmatizer(True)
preproc.PorterStemmer(True)
preproc.good_preproc()
preproc_data = preproc.tweets

/var/folders/cc/m1zdnv7157ncsc484c8h7fvr0000gn/T/ipykernel_19423/1090301464.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets = tweets.append(df)
/var/folders/cc/m1zdnv7157ncsc484c8h7fvr0000gn/T/ipykernel_19423/1090301464.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets = tweets.append(df)
/var/folders/cc/m1zdnv7157ncsc484c8h7fvr0000gn/T/ipykernel_19423/1090301464.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets = tweets.append(df)
[nltk_data] Downloading package wordnet to /Users/banji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/banji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloadi

In [28]:
preproc_data

,tweets,category,just_tokenization,PorterStemmer,WordNetLemmatizer,WordNetLemmatizer+misspellings,PorterStemmer+misspellings,good_preproc
0,How unhappy some dogs like it though,-1,"[How, unhappy, some, dogs, like, it, though]","[how, unhappi, some, dog, like, it, though]","[How, unhappy, some, dog, like, it, though]","[How, unhappy, some, dog, like, it, though]","[how, unhappi, some, dog, like, it, though]","[unhappy, dogs, like, though]"
1,talking to my over driver about where I'm goin...,-1,"[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, i, ...","[talking, to, my, over, driver, about, where, ...","[talking, to, my, over, driver, about, where, ...","[talk, to, my, over, driver, about, where, i, ...","[talk, my, driver, where, 'm, goingh, said, 'd..."
2,Does anybody know if the Rand's likely to fall...,-1,"[Does, anybody, know, if, the, Rand, 's, likel...","[doe, anybodi, know, if, the, rand, 's, like, ...","[Does, anybody, know, if, the, Rand, 's, likel...","[Does, anybody, know, if, the, Rand, 's, likel...","[doe, anybodi, know, if, the, rand, 's, like, ...","[doe, anybodi, know, the, rand, likely, fall, ..."
3,I miss going to gigs in Liverpool unhappy,-1,"[I, miss, going, to, gigs, in, Liverpool, unha...","[i, miss, go, to, gig, in, liverpool, unhappi]","[I, miss, going, to, gig, in, Liverpool, unhappy]","[I, miss, going, to, gig, in, Liverpool, unhappy]","[i, miss, go, to, gig, in, liverpool, unhappi]","[miss, to, gig, Liverpool, unhappi]"
4,There isnt a new Riverdale tonight ? unhappy,-1,"[There, isnt, a, new, Riverdale, tonight, ?, u...","[there, isnt, a, new, riverdal, tonight, ?, un...","[There, isnt, a, new, Riverdale, tonight, ?, u...","[There, isnt, a, new, Riverdale, tonight, ?, u...","[there, isnt, a, new, riverdal, tonight, ?, un...","[isnt, new, riverdal, tonight, unhappy]"
...,...,...,...,...,...,...,...,...
2740,Supreme Court shoots down govt bid to put spor...,0,"[Supreme, Court, shoots, down, govt, bid, to, ...","[suprem, court, shoot, down, govt, bid, to, pu...","[Supreme, Court, shoot, down, govt, bid, to, p...","[Supreme, Court, shoot, down, govt, bid, to, p...","[suprem, court, shoot, down, govt, bid, to, pu...","[suprem, court, shoot, govt, bid, put, sport, ..."
2741,"Historian Ram Guha, IDFC official Vikram Limay...",0,"[Historian, Ram, Guha, ,, IDFC, official, Vikr...","[historian, ram, guha, ,, idfc, offici, vikram...","[Historian, Ram, Guha, ,, IDFC, official, Vikr...","[Historian, Ram, Guha, ,, IDFC, official, Vikr...","[historian, ram, guha, ,, idfc, offici, vikram...","[historian, ram, guha, IDFC, offici, vikram, l..."
2742,Supreme Court names former CAG as head of 4-me...,0,"[Supreme, Court, names, former, CAG, as, head,...","[suprem, court, name, former, cag, as, head, o...","[Supreme, Court, name, former, CAG, a, head, o...","[Supreme, Court, name, former, CAG, a, head, o...","[suprem, court, name, former, cag, as, head, o...","[suprem, court, name, former, cag, head, 4-mem..."
2743,Court summons CM suspended BJP MP as accused i...,0,"[Court, summons, CM, suspended, BJP, MP, as, a...","[court, summon, cm, suspend, bjp, mp, as, accu...","[Court, summons, CM, suspended, BJP, MP, a, ac...","[Court, summons, CM, suspended, BJP, MP, a, ac...","[court, summon, cm, suspend, bjp, mp, as, accu...","[court, summon, suspended, bjp, as, accus, cri..."


In [29]:
preproc_data.category.value_counts()

 0    1048
-1     859
 1     838
Name: category, dtype: int64

In [30]:
class TO_VEC:
    def __init__(self, data_frame):
        self.text = data_frame
        self.vectors = pd.DataFrame()
        self.vectorizers = {}

    def to_bin_counts(self):
        coun_vect = CountVectorizer(binary=True)
        for i in range(2, self.text.shape[1]):
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,i])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = coun_vect.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["bin_vec_{}".format(name)] = list(count_array)
            self.vectorizers["bin_vec_{}".format(name)] = coun_vect

    def to_TFIDF(self):
        vectorizer = TfidfVectorizer()
        for i in range(2, self.text.shape[1]):
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,i])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = vectorizer.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["TFIDF_{}".format(name)] = list(count_array)
            self.vectorizers["TFIDF_{}".format(name)] = vectorizer

    def to_word_counts(self):
        coun_vect = CountVectorizer()
        for i in range(2, self.text.shape[1]):
            text = pd.DataFrame(map(lambda x: ' '.join(x), self.text.iloc[:,i])).iloc[:, 0].tolist()
            name = self.text.iloc[:,i].name
            count_matrix = coun_vect.fit_transform(text)
            count_array = count_matrix.toarray()
            self.vectors["word_counts_{}".format(name)] = list(count_array)
            self.vectorizers["word_counts_{}".format(name)] = coun_vect


    #Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data
    def to_word2vec(self):
      word2vec = gensim.downloader.load("glove-twitter-25")
      mean = np.mean(word2vec.vectors, 0)
      std = np.std(word2vec.vectors, 0)
      for i in range(2, self.text.shape[1]):
        text = self.text.iloc[:,i].tolist()
        array = []
        name = self.text.iloc[:,i].name
        for i in range(len(text)):
          line = text[i]
          filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
          embeddings = np.mean([(word2vec.get_vector(w) - mean)/std for w in filtered_line if w in word2vec],axis=0)
          if len(embeddings.shape) == 0:
            embeddings = np.zeros((1, word2vec.vector_size))
          else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
              embeddings = embeddings.reshape(1, -1)
          array.append(embeddings.reshape(25))
        self.vectors["word2vec_{}".format(name)] = array
        # self.vectorizers["word2vec_{}".format(name)] = coun_vect


Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.



In [31]:
vectors = TO_VEC(preproc_data)
vectors.to_bin_counts()
vectors.to_TFIDF()
vectors.to_word_counts()
vectors.to_word2vec()

In [32]:
vectors.vectors

,bin_vec_just_tokenization,bin_vec_PorterStemmer,bin_vec_WordNetLemmatizer,bin_vec_WordNetLemmatizer+misspellings,bin_vec_PorterStemmer+misspellings,bin_vec_good_preproc,TFIDF_just_tokenization,TFIDF_PorterStemmer,TFIDF_WordNetLemmatizer,TFIDF_WordNetLemmatizer+misspellings,...,word_counts_WordNetLemmatizer,word_counts_WordNetLemmatizer+misspellings,word_counts_PorterStemmer+misspellings,word_counts_good_preproc,word2vec_just_tokenization,word2vec_PorterStemmer,word2vec_WordNetLemmatizer,word2vec_WordNetLemmatizer+misspellings,word2vec_PorterStemmer+misspellings,word2vec_good_preproc
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.2853099, -0.09511143, 0.121866025, 0.244779...","[0.678889, -0.029011795, 0.37241903, 0.1751234...","[0.40553612, -0.009428509, 0.10449699, 0.23827...","[0.40553612, -0.009428509, 0.10449699, 0.23827...","[0.678889, -0.029011795, 0.37241903, 0.1751234...","[0.11180677, -0.25933897, 0.050498232, 0.11771..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.5349254, 0.22520572, 0.53480244, -0.2685251...","[0.14421621, 0.04486893, -0.043388907, -0.2901...","[0.5349254, 0.22520572, 0.53480244, -0.2685251...","[0.5349254, 0.22520572, 0.53480244, -0.2685251...","[0.14421621, 0.04486893, -0.043388907, -0.2901...","[-0.057604894, 0.020860864, -0.057774942, -0.4..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.41505033, 0.3681989, -0.41270638, 0.0710058...","[0.46806738, 0.30985337, -0.32477564, 0.217874...","[0.4595495, 0.42750767, -0.4832863, 0.08044413...","[0.4595495, 0.42750767, -0.4832863, 0.08044413...","[0.46806738, 0.30985337, -0.32477564, 0.217874...","[0.5824763, 0.45816827, -0.7259443, 0.01275363..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0

# TRAIN

In [33]:
def train(model, data, target):
    accuracy_table = pd.DataFrame(columns=list(data.keys()))
    models = {}
    for i in range(data.shape[1]):
        default_model = model
        X = data.iloc[:,i]
        name = X.name
        y = target
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    shuffle = True)
        X_train = pd.DataFrame(X_train.tolist())
        X_test = pd.DataFrame(X_test.tolist())
        y_train = y_train.tolist()
        y_test = y_test.tolist()
        
        default_model.fit(X_train, y_train)
        predict = default_model.predict(X_test)
        accuracy_table.iloc[:,i] = [accuracy_score(predict, y_test)]
        models[name] = default_model
        
    return accuracy_table, models

In [34]:
from sklearn import svm
svc = svm.SVC()
accuracy_table_svm, models_svm = train(svc, vectors.vectors, preproc_data.category)

In [35]:
accuracy_table_svm

,bin_vec_just_tokenization,bin_vec_PorterStemmer,bin_vec_WordNetLemmatizer,bin_vec_WordNetLemmatizer+misspellings,bin_vec_PorterStemmer+misspellings,bin_vec_good_preproc,TFIDF_just_tokenization,TFIDF_PorterStemmer,TFIDF_WordNetLemmatizer,TFIDF_WordNetLemmatizer+misspellings,...,word_counts_WordNetLemmatizer,word_counts_WordNetLemmatizer+misspellings,word_counts_PorterStemmer+misspellings,word_counts_good_preproc,word2vec_just_tokenization,word2vec_PorterStemmer,word2vec_WordNetLemmatizer,word2vec_WordNetLemmatizer+misspellings,word2vec_PorterStemmer+misspellings,word2vec_good_preproc
0,0.907104,0.907104,0.907104,0.907104,0.907104,0.897996,0.899818,0.907104,0.901639,0.901639,...,0.903461,0.903461,0.905282,0.897996,0.82878,0.783242,0.82878,0.82878,0.783242,0.781421


In [36]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
accuracy_table_logreg, models_logreg = train(lr, vectors.vectors, preproc_data.category)

/Users/banji/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/banji/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [37]:
accuracy_table_logreg

,bin_vec_just_tokenization,bin_vec_PorterStemmer,bin_vec_WordNetLemmatizer,bin_vec_WordNetLemmatizer+misspellings,bin_vec_PorterStemmer+misspellings,bin_vec_good_preproc,TFIDF_just_tokenization,TFIDF_PorterStemmer,TFIDF_WordNetLemmatizer,TFIDF_WordNetLemmatizer+misspellings,...,word_counts_WordNetLemmatizer,word_counts_WordNetLemmatizer+misspellings,word_counts_PorterStemmer+misspellings,word_counts_good_preproc,word2vec_just_tokenization,word2vec_PorterStemmer,word2vec_WordNetLemmatizer,word2vec_WordNetLemmatizer+misspellings,word2vec_PorterStemmer+misspellings,word2vec_good_preproc
0,0.903461,0.908925,0.905282,0.905282,0.908925,0.905282,0.910747,0.918033,0.908925,0.908925,...,0.901639,0.901639,0.901639,0.901639,0.82878,0.765027,0.830601,0.830601,0.765027,0.763206


In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
# parameters = {'max_depth':[3,10]} #лучший 10
# gsCV = GridSearchCV(gbc, parameters)
accuracy_table_gbc, models_gbc = train(gbc, vectors.vectors, preproc_data.category)

In [39]:
accuracy_table_gbc

,bin_vec_just_tokenization,bin_vec_PorterStemmer,bin_vec_WordNetLemmatizer,bin_vec_WordNetLemmatizer+misspellings,bin_vec_PorterStemmer+misspellings,bin_vec_good_preproc,TFIDF_just_tokenization,TFIDF_PorterStemmer,TFIDF_WordNetLemmatizer,TFIDF_WordNetLemmatizer+misspellings,...,word_counts_WordNetLemmatizer,word_counts_WordNetLemmatizer+misspellings,word_counts_PorterStemmer+misspellings,word_counts_good_preproc,word2vec_just_tokenization,word2vec_PorterStemmer,word2vec_WordNetLemmatizer,word2vec_WordNetLemmatizer+misspellings,word2vec_PorterStemmer+misspellings,word2vec_good_preproc
0,0.899818,0.899818,0.901639,0.901639,0.905282,0.892532,0.897996,0.894353,0.901639,0.901639,...,0.903461,0.899818,0.899818,0.89071,0.825137,0.770492,0.830601,0.82878,0.770492,0.766849


# Similarity

top-10 most similar pair of tweets

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

matrix_similarity = []
data_1 = vectors.vectors.TFIDF_good_preproc
data_2 = vectors.vectors.TFIDF_WordNetLemmatizer

matrix_similarity = cosine_similarity(list(data_1))
cos_sim = []
for i in range(len(matrix_similarity)):
    for j in range(len(matrix_similarity[i])):
        if (i == j): matrix_similarity[i][j] = -np.inf
    a = list([i, np.argmax(matrix_similarity[i]),max(matrix_similarity[i])])
    cos_sim.append(a)

cos_sim.sort(key = lambda x: -x[2])

cos_sim[:10]

[[159, 214, 1.0000000000000002],
 [179, 851, 1.0000000000000002],
 [214, 159, 1.0000000000000002],
 [437, 711, 1.0000000000000002],
 [574, 159, 1.0000000000000002],
 [608, 679, 1.0000000000000002],
 [654, 690, 1.0000000000000002],
 [679, 608, 1.0000000000000002],
 [690, 654, 1.0000000000000002],
 [711, 437, 1.0000000000000002]]

In [ ]:
vectors.text.tweets[159], vectors.text.tweets[214]

('I miss you unhappy ', 'i miss them unhappy ')

In [ ]:
vectors.text.tweets[711], vectors.text.tweets[437]

('babies unhappy  ', 'baby unhappy  ')

In [ ]:
def predict_class(text, model, vectorizer):
    vectors = vectorizer.transform(text)
    return model.predict(vectors)

In [ ]:
predict_class(["i love you", "fuck", 'abra abrab ab', 'bad bad bad bad bad', 'thank you very much you helped me'], gbc, vectors.vectorizers['TFIDF_good_preproc'])

array([1, 0, 0, 0, 0])

In [ ]:

# def plot_conf_matrix(true, pred, cat=['-1', '0', '1']):
#     sns.heatmap(data=confusion_matrix(true, pred), 
#             annot=True, fmt="d", cbar=False, xticklabels=cat, yticklabels=cat)


# plot_conf_matrix(y_test, tfidf_pred_data)

# accuracy_score(y_test, tfidf_pred_data)

In [ ]:
#понадобится для nltk.download()
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()